In [8]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(400_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
import utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [9]:
importlib.reload(utils)
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [10]:
air_joined = pd.read_parquet('airtable_joined.parquet')

In [11]:
# ran_operators = [int(filename.split('_')[0]) for filename in os.listdir('speedmaps') if filename[0] != '.']
ran_operators = []

In [12]:
pbar = tqdm()

0it [00:00, ?it/s]

In [17]:
agency = 290
analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables

In [19]:
rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:38: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')


found_parquet
6591 scheduled trips out of 6694 have no shape, dropping


AssertionError: >10% of trips have no shape!

In [ ]:
# for agency in air_joined.calitp_itp_id.unique():
for agency in [290]:
    not_ran_operators = []
    if agency in ran_operators:
        print(f'already ran: {agency}')
        continue
    analysis_date = dt.date(2022, 3, 23) ##wednesday, new tables
    day = str(analysis_date.day).zfill(2)
    print(f'calculating for agency: {agency}...')
    try:
        rt_day = rt.OperatorDayAnalysis(agency, analysis_date, pbar)
        rt_day.set_filter(start_time='15:00', end_time='19:00')
        m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
        shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
                                                 f'{utils.GCS_FILE_PATH}segment_speed_views',
                                                f'{agency}_02_{day}_pm_peak'
                                                )
        rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        # m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
        # rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        delay_to_parquet = rt_day.endpoint_delay_view.copy()
        delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
        delay_to_parquet = delay_to_parquet >> select(-_.delay)
        delay_to_parquet.to_parquet(f'{utils.GCS_FILE_PATH}endpoint_delay_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
        print(f'complete for agency: {agency}')
    except Exception as e:
        print(f'rt failed for agency {agency}')
        not_ran_operators += [agency]
        print(e)

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', size = [1300, 700])
shared_utils.utils.geoparquet_gcs_export(rt_day.stop_segment_speed_view,
                                         f'{utils.GCS_FILE_PATH}segment_speed_views',
                                        f'{agency}_02_{day}_pm_peak'
                                        )
rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
# m.save(f'./speedmaps/{agency}_02_{day}_pm_peak.html')
# rt_day.stop_segment_speed_view.to_parquet(f'{utils.GCS_FILE_PATH}segment_speed_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling
delay_to_parquet = rt_day.endpoint_delay_view.copy()
delay_to_parquet['delay_seconds'] = delay_to_parquet.delay.map(lambda x: x.seconds)
delay_to_parquet = delay_to_parquet >> select(-_.delay)
delay_to_parquet.to_parquet(f'{utils.GCS_FILE_PATH}endpoint_delay_views/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling

In [7]:
# rt_day.rt_trips.to_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{agency}_02_{day}_pm_peak.parquet') ## early step towards scaling

### Ongoing issues
* 358 Union City too many trips without shape?

In [8]:
# air_joined['url'] = air_joined.apply(lambda x:
#                     f'http://docs.calitp.org/data-analyses/rt_delay/speedmaps/{x.calitp_itp_id}_02_08_pm_peak.html',
#                                     axis = 1)

# air_joined.to_csv('linked.csv')